# Craw PTT

In [1]:
from selenium.common.exceptions import NoSuchElementException

In [2]:
from selenium import webdriver
import time

In [3]:
url='https://www.ptt.cc/bbs/Cross_Life/index.html'

In [4]:
# 指定使用 chromedriver.exe 來執行瀏覽器作業
driver = webdriver.Chrome('chromedriver.exe')

# 跳至指定網址
driver.get(url)
# 設定cookie這樣可以進入男女版
driver.add_cookie({
    'name' : 'over18',
    'value' : '1'
})
# seleium有點bug, 最好每次到新網頁先將瀏覽器maxmize到畫面最前面
driver.minimize_window()
driver.maximize_window()

In [5]:
# 定義新的類別來裝我們想要的東西
class CrawResult():
    
    # 設定實體化物件的時候將我們想要的內容裝進去
    def __init__(self, author, title, block, time, content):
        self.author=author
        self.title=title
        self.block=block
        self.time=time
        self.content=content
    
    # 定義方法將存放的內容印出
    def showInfo(self):
        print('作者:', self.author)
        print('標題:', self.title)
        print('看板:', self.block)
        print('日期:', self.time)
        print('內文:', self.content)

In [6]:
def get_article():
    '''
    craw the information in current page of driver.
    the information include: author, title, block, time, content
    
    return CrawResult()
    '''
    from selenium.common.exceptions import NoSuchElementException
    
    try:
        # 抓取作者
        author = driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/span[2]')
        author_text = author.text
        # 抓取標題
        title = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]')
        title_text = title.text
        # 抓取看板
        block = driver.find_element_by_xpath('//*[@id="main-content"]/div[2]/span[2]')
        block_text = block.text
        # 抓取日期
        time = driver.find_element_by_xpath('//*[@id="main-content"]/div[4]/span[2]')
        time_text = time.text

        # 抓取內文, 想法是由於內文並沒有標籤, 先將其他有標籤的物件刪除再抓取內文.
        # 先將出現在目標標籤內的其他標籤class定義出來.
        lst = ['article-metaline-right', 'article-metaline', 'f2', 'push', 'f3']
        # 透過javascript語法將這些多餘的標籤刪除
        # 由於有未定義的class, 因此會出現exception, 於是連續執行5次刪除的動作才能刪乾淨
        for _ in range(5):
            for i in lst:
                try:
                    driver.execute_script("a=document.getElementsByClassName(\'{}\')".format(i))
                    driver.execute_script('for (i=0;i<=a.length;i++){a[i].parentNode.removeChild(a[i])}')
                except :
                    pass
        # 將多餘的標籤刪除後抓取內文
        content = driver.find_element_by_xpath('//*[@id="main-content"]')
        content_text = content.text

        # 將抓取到的資料丟進物件中回傳
        craw_result = CrawResult(author_text, title_text, block_text, time_text, content_text)
    # 版規是抓不到作者的, 就無視這個頁面
    except NoSuchElementException as e:
        craw_result = None
        
    return craw_result

In [7]:
# 定義空的list以接收每個頁面的資料
craw_results=[]

if len(driver.current_url.split('/'))==5:
    print('please enter a block to craw pages.')
else:
    # 如果內容裡面有多個文章, 就一個一個進去抓資料, 如果網址是單一文章就直接取出
    try:
        # 抓取每篇文章的網址 
        # 由於每個網址位於不同的標籤中, 因此先取出不同的網頁物件存在list中
        r_ents=[]
        for i in range(2, 21):
            # 有的時候會有分隔線或是已刪除文章, 要記得把exception catch出來
            try:
                xpath = '//*[@id="main-container"]/div[2]/div[{}]/div[2]/a'.format(i)
                r_ents.append(driver.find_element_by_xpath(xpath))
            except NoSuchElementException:
                print('NoSuchElementException occur:', xpath)
                
        # 迴圈針對每一個物件取出連結網址
        ents = [ent.get_attribute('href') for ent in r_ents]

        # 連到不同的網址中取文章內容
        for ent in ents:
            driver.execute_script("window.location.assign(\'{}\')".format(ent))
            craw_result = get_article()
            craw_results.append(craw_result)
            time.sleep(5)
    # 如果不是在文章頁面執行的話就直接取出文章內容
    except : 
        craw_result = get_article()
        craw_results.append(craw_result)

In [8]:
# 將取得的資料印出
for result in craw_results:
    print(result.showInfo())
    print("============================")

作者: saltlake (SaltLake)
標題: 標題[問題] 大戶人家
看板: Cross_Life
日期: Fri Feb  9 05:14:35 2018
內文:  文獻裡面，古時候所謂大戶人家有所謂王半城或者賴半街

 那類，富有到全城有半數的土地都他家的，或者某市鎮上

 的某條繁華商業街，有一半的土地是他家的。或者說，某

 城鎮地方的某一類甚至幾類產業都是某家的。例如林記米

 店可以包括全鎮甚至全鄉，或者本縣的馬車行和碼頭苦力

 等運輸與搬運業，都被某人或某家族掌握等等。

   大陸改革開放之後至今，民間財富迅速累積，有沒有出現

 這類古時候的現象?或者說，現在大陸所謂的大戶人家，可以

 大到啥地步?

--
None
作者: saltlake (SaltLake)
標題: 標題[問題] 大陸過年的特色活動
看板: Cross_Life
日期: Tue Feb 13 10:19:32 2018
內文:   傳統活動之一是放鞭炮，但是現在似乎被禁了?

  另外就是家人圍爐看央視的春晚，但自改革至今

  ，各種新電台和網路興起等因素，春晚的光芒(素

  質)似乎大為下降，還是大陸多數人會做的過年活

  動之一嗎?

  除此之外，現在被認同是過年特色活動，而且還是

  多數大陸人歡迎或會做的活動，有啥?

--
None
作者: jaymin (JAY)
標題: 標題[請益]象印電子鍋-海外中國版
看板: Cross_Life
日期: Wed Feb 21 10:15:42 2018
內文: 各位好，不知道這種事物放在這適不適當，

也請大家見諒了。




小弟想要賣一台全新的象印電子鍋 NP-HIH10 5CUP

https://imgur.com/a/QAc7z

https://world.taobao.com/item-amp/547545562683.htm

由於我爸媽一時疏忽，非常開心的去日本抱了一台電子鍋回來。

結果開箱玩才發現電壓規格於台灣不同，

是中國版AC220 V的電壓的飯鍋。嗚嗚嗚...




接下來是價錢的部份了，日幣58400元，折合台幣為15709元。

在淘寶有看到此產品為，人民幣3350，折合台幣為15407元。

如果有人想要，我可以以台幣15000元賣出。

或著價錢怎樣都可以再談

In [9]:
driver.close()